In [1]:
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (16, 9)
plt.style.use('ggplot')
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from IPython.display import Image as PImage
from subprocess import check_call
from PIL import Image, ImageDraw, ImageFont

# Pre procesamiento de los datos


## Postulantes

In [2]:
postulantes_educacion_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_desde_15_Abril = pd.read_csv('Datos/Fiuba_desde_15_Abril/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_educacion_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_hasta_15_abril = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_educacion_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_genero_y_edad_original = pd.read_csv('Datos/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')

In [12]:
postulantes_genero_y_edad = pd.concat([postulantes_genero_y_edad_desde_15_Abril,postulantes_genero_y_edad_hasta_15_abril,postulantes_genero_y_edad_original], axis=0)
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates('idpostulante')

In [19]:
postulantes_educacion = pd.concat([postulantes_educacion_desde_15_Abril,postulantes_educacion_hasta_15_abril,postulantes_educacion_original], axis=0)
postulantes_educacion = postulantes_educacion.drop_duplicates('idpostulante')

In [24]:
#Renombro las columnas a un nombre más apropiado

postulantes_educacion = postulantes_educacion.rename(columns={"nombre":"educacion"})

In [25]:
#Hay algun dato nulo?

postulantes_educacion.isnull().any()

idpostulante    False
educacion       False
estado          False
dtype: bool

In [26]:
#Para cada postulante, me interesa quedarme solamente con la mayor de sus educaciones

postulantes_educacion['Cod_edu'] = np.array([0 if e == 'Secundario' else 
                                             1 if e == 'Otro' else 
                                             2 if e == 'Terciario/Técnico' else 
                                             3 if e == 'Universitario' else 
                                             4 if e == 'Posgrado' else 
                                             5 if e == 'Master' else 6 for e in postulantes_educacion.educacion])

postulantes_educacion = postulantes_educacion.sort_values(['idpostulante','Cod_edu'], ascending = False).drop_duplicates('idpostulante')

In [27]:
postulantes_educacion.head()

,idpostulante,educacion,estado,Cod_edu
53092,zvxzzL6,Secundario,Graduado,0
118722,zvxzvD5,Terciario/Técnico,En Curso,2
60153,zvxzrWm,Otro,Graduado,1
50863,zvxzmED,Secundario,Graduado,0
211836,zvxzjoK,Terciario/Técnico,En Curso,2


In [30]:
#Convierto la columna fechanacimiento a formato datetime. 

postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'], errors='coerce')

C:\Users\Mariano\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [32]:
#Vamos ahora a agregar una columna adicional al dataframe, de modo que podamos observar la edad de cada persona

postulantes_genero_y_edad['edad'] = ((pd.datetime.today() - postulantes_genero_y_edad['fechanacimiento']).dt.days/365)
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].fillna(0)
postulantes_genero_y_edad['edad'] = postulantes_genero_y_edad['edad'].astype(int)
postulantes_genero_y_edad.head(10)

C:\Users\Mariano\Anaconda2\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\Mariano\Anaconda2\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\Mariano\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_ind

,idpostulante,fechanacimiento,sexo,edad
0,6MM,1985-01-01,MASC,33
1,ZX1,NaT,NO_DECLARA,0
2,YZP,1961-05-01,MASC,57
3,ebE,1952-07-07,MASC,65
4,N1x,NaT,NO_DECLARA,0
5,8mW,NaT,NO_DECLARA,0
6,52aw,NaT,NO_DECLARA,0
7,NAjM,1962-06-09,FEM,56
8,a6OE,NaT,NO_DECLARA,0
9,ZjlZ,1970-01-25,MASC,48


In [33]:
#Reviso la edad de las personas que cumplen años el 29/02

postulantes_nacidos_en_febrero = postulantes_genero_y_edad.loc[postulantes_genero_y_edad['fechanacimiento'].dt.month == 2]
postulantes_nacidos_en_febrero.loc[postulantes_nacidos_en_febrero['fechanacimiento'].dt.day == 29].sort_values('fechanacimiento',ascending = False).head(10)

,idpostulante,fechanacimiento,sexo,edad
232070,EzeN2Q4,2000-02-29,FEM,18
213258,96XkKlw,2000-02-29,FEM,18
297009,JBraa0E,2000-02-29,MASC,18
259430,ekOwAWW,2000-02-29,FEM,18
13782,VNr1dej,2000-02-29,MASC,18
176574,1QP82xR,2000-02-29,FEM,18
165756,Yj5p1GX,2000-02-29,MASC,18
186798,Bm5dkzR,2000-02-29,MASC,18
194211,3NP3dAJ,2000-02-29,FEM,18
197120,6rP38R3,2000-02-29,FEM,18


In [34]:
#Merge de los datos de los postulantes

postulantes = pd.merge(postulantes_genero_y_edad,postulantes_educacion,how='left',on='idpostulante')
postulantes['educacion'] = postulantes['educacion'].fillna('Ninguna')
postulantes = postulantes.drop(['fechanacimiento','Cod_edu'],axis=1)
postulantes.head()

,idpostulante,sexo,edad,educacion,estado
0,6MM,MASC,33,Ninguna,NaN
1,ZX1,NO_DECLARA,0,Ninguna,NaN
2,YZP,MASC,57,Master,En Curso
3,ebE,MASC,65,Universitario,Graduado
4,N1x,NO_DECLARA,0,Ninguna,NaN
5,8mW,NO_DECLARA,0,Ninguna,NaN
6,52aw,NO_DECLARA,0,Ninguna,NaN
7,NAjM,FEM,56,Ninguna,NaN
8,a6OE,NO_DECLARA,0,Ninguna,NaN
9,ZjlZ,MASC,48,Master,En Curso


In [38]:
postulantes.drop_duplicates('idpostulante').shape

(504407, 5)

## Vistas

In [ ]:
vistas = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_3_vistas.csv')

In [29]:
#Vistazo a la estructura del dataframe

vistas.head()

,idAviso,timestamp,idpostulante
0,1112400775,2018-04-06T01:43:47.445-0400,kPjr2K0
1,1112400775,2018-04-06T01:43:48.330-0400,kPjr2K0
2,1112426664,2018-04-06T01:43:50.042-0400,GNOwZmb
3,1112426664,2018-04-06T01:43:51.171-0400,GNOwZmb
4,1112445738,2018-04-06T01:43:46.371-0400,lDLxajz


In [30]:
#El dataframe posee 5758686 registros

vistas.shape

(5758686, 3)

In [31]:
#No tenemos registros nulos

vistas.isnull().any()

idAviso         False
timestamp       False
idpostulante    False
dtype: bool

In [32]:
#En este dataframe tenemos 11223 avisos únicos 

vistas['idAviso'].value_counts().size

11223

In [34]:
#Y de los postulantes del dataset, 272526 han visto al menos un aviso

vistas['idpostulante'].value_counts().size

272526

In [35]:
#Modifico el nombre de la columna timestamp y en su lugar pongo un nombre más descriptivo.>

vistas = vistas.rename(columns = {'timestamp':'fechahoravisualizacion'})

In [36]:
#Ordeno la informacion sobre la fecha y hora de visualizaciones, modificando su formato y agregando columnas que me permitan
#analizar de forma individual la fecha y la hora.

vistas['fechahoravisualizacion'] = pd.to_datetime(vistas['fechahoravisualizacion'])
vistas['fechavisualizacion'] = vistas['fechahoravisualizacion'].dt.date
vistas['horavisualizacion'] = vistas['fechahoravisualizacion'].dt.time
vistas.sort_values('fechavisualizacion').head(10)

,idAviso,fechahoravisualizacion,idpostulante,fechavisualizacion,horavisualizacion
1139203,1112414906,2018-03-22 22:25:05.755,8GmYkR,2018-03-22,22:25:05.755000
2796318,1112115684,2018-03-22 16:29:35.883,5mlDebG,2018-03-22,16:29:35.883000
2796319,1112417734,2018-03-22 16:29:36.751,xkv6aEK,2018-03-22,16:29:36.751000
2796320,1112396603,2018-03-22 16:29:41.338,EzZbro6,2018-03-22,16:29:41.338000
2796321,1112280484,2018-03-22 16:29:41.417,NE8vdL,2018-03-22,16:29:41.417000
2796322,1112371748,2018-03-22 16:29:41.689,8MZk5W3,2018-03-22,16:29:41.689000
2796323,1112403133,2018-03-22 16:29:41.705,8Mabwwz,2018-03-22,16:29:41.705000
2796317,1112353401,2018-03-22 16:29:31.254,EzeoZ2b,2018-03-22,16:29:31.254000
2796324,1000851124,2018-03-22 16:29:41.719,0zB8NJ1,2018-03-22,16:29:41.719000
2796326,1112308737,2018-03-22 16:29:44.367,DrXBrkZ,2018-03-22,16:29:44.367000


## Postulaciones

In [ ]:
postulaciones = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv')

In [37]:
#Vistazo a la estructura del dataframe

postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112248724,NjlD,2018-01-19 07:39:16
1,1112286523,ZaO5,2018-01-24 15:07:39
2,1112272060,ZaO5,2018-01-24 15:20:10
3,1112288401,ZaO5,2018-01-26 08:37:04
4,1112300563,ZaO5,2018-01-30 13:35:48


In [38]:
#Tenemos 4909641 registros

postulaciones.shape

(4909641, 3)

In [39]:
#No hay registros nulos

postulaciones.isnull().any()

idaviso             False
idpostulante        False
fechapostulacion    False
dtype: bool

In [40]:
#Tenemos 20065 avisos con al menos una postulacion

postulaciones['idaviso'].value_counts().size

20065

In [41]:
#Hay postulantes que no han realizado ninguna postulacion

postulaciones['idpostulante'].value_counts().size

238482

In [304]:
##Ordeno la informacion sobre la fecha y hora de postulación, modificando su formato y agregando columnas que me permitan
#analizar de forma individual la fecha y la hora.

postulaciones['fechapostulacion'] = pd.to_datetime(postulaciones['fechapostulacion'])
postulaciones = postulaciones.rename(columns={'fechapostulacion':'fechahorapostulacion'})
postulaciones['fechapostulacion'] = postulaciones['fechahorapostulacion'].dt.date
postulaciones['horapostulacion'] = postulaciones['fechahorapostulacion'].dt.time

postulaciones.sort_values('fechapostulacion').head(10)

,idaviso,idpostulante,fechahorapostulacion,fechapostulacion,horapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34,2018-01-15,16:22:34
736839,1112261550,qekqr4b,2018-01-15 19:00:07,2018-01-15,19:00:07
2524985,1112145604,N5Doal,2018-01-15 11:48:56,2018-01-15,11:48:56
2524984,1112239063,N5Doal,2018-01-15 11:44:57,2018-01-15,11:44:57
736893,1112257631,PmV48YJ,2018-01-15 10:12:34,2018-01-15,10:12:34
736894,1112261524,PmV48YJ,2018-01-15 16:26:58,2018-01-15,16:26:58
736941,1112245598,KBAvVxX,2018-01-15 22:41:33,2018-01-15,22:41:33
736942,1112247307,KBAvVxX,2018-01-15 22:42:12,2018-01-15,22:42:12
736943,1111936969,KBAvVxX,2018-01-15 22:42:56,2018-01-15,22:42:56
736944,1112251166,KBAvVxX,2018-01-15 22:43:08,2018-01-15,22:43:08


## Avisos

In [30]:
avisos_online = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv')
avisos_detalle = pd.read_csv('Datos/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv')

In [31]:
avisos_detalle.shape

(18299, 11)

In [305]:
#Vistazo a la estructura del dataframe

avisos_online.head()

,idaviso
0,1112355872
1,1112335374
2,1112374842
3,1111984070
4,1111822480


In [306]:
#Existen 5028 avisos online

avisos_online.shape

(5028, 1)

In [307]:
#No hay registros nulos

avisos_online.isnull().any()

idaviso    False
dtype: bool

In [308]:
#Tampoco hay ids repetidos

avisos_online['idaviso'].value_counts().size

5028

In [309]:
#Vistazo a la estructura del dataframe

avisos_detalle.head()

,idaviso,idpais,titulo,descripcion,nombre_zona,ciudad,mapacalle,tipo_de_trabajo,nivel_laboral,nombre_area,denominacion_empresa
0,8725750,1,VENDEDOR/A PROVINCIA DE SANTA FE,<p><strong><strong>Empresa:</strong></strong> ...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Comercial,VENTOR
1,17903700,1,Enfermeras,<p>Solicitamos para importante cadena de farma...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Salud,Farmacias Central Oeste
2,1000150677,1,Chofer de taxi,<p>TE GUSTA MANEJAR? QUERES GANAR PLATA HACIEN...,Capital Federal,NaN,Empedrado 2336,Full-time,Senior / Semi-Senior,Transporte,FAMITAX SRL
3,1000610287,1,CHOFER DE CAMIONETA BAHIA BLANCA - PUNTA ALTA,<p><strong>Somos una empresa multinacional que...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Transporte,Wurth Argentina S.A
4,1000872556,1,Operarios de Planta - Rubro Electrodomésticos,<p><strong>OPERARIOS DE PLANTA</strong></p><p>...,Gran Buenos Aires,NaN,NaN,Full-time,Senior / Semi-Senior,Producción,ELECTRO OUTLET SRL


In [310]:
#Existen 13534 avisos contra los 5028 que teniamos en avisos_online. Significa que los restantes 8506 están inactivos?

avisos_detalle.shape

(13534, 11)

In [311]:
#Tenemos nulos en los campos ciudad, mapa y denominacion_empresa

avisos_detalle.isnull().any()

idaviso                 False
idpais                  False
titulo                  False
descripcion             False
nombre_zona             False
ciudad                   True
mapacalle                True
tipo_de_trabajo         False
nivel_laboral           False
nombre_area             False
denominacion_empresa     True
dtype: bool

In [312]:
#13487 no tienen registrada la ciudad

avisos_detalle.loc[avisos_detalle['ciudad'].isnull()].shape

(13487, 11)

In [313]:
#Cinco avisos no tienen registrada la empresa que lleva a cabo la búsqueda

avisos_detalle.loc[avisos_detalle['denominacion_empresa'].isnull()].shape

(5, 11)

In [314]:
#Todos los avisos son del mismo país

avisos_detalle['idpais'].value_counts()

1    13534
Name: idpais, dtype: int64

In [315]:
#Zonas entre las que se reparten los avisos, podemos corroborar que todos los avisos 
#corresponden a Buenos Aires como dice la consigna

avisos_detalle['nombre_zona'].value_counts()

Gran Buenos Aires              12654
Capital Federal                  876
GBA Oeste                          2
Buenos Aires (fuera de GBA)        2
Name: nombre_zona, dtype: int64

In [316]:
postulantes_educacion.educacion.value_counts()

Secundario           110256
Universitario        104295
Terciario/Técnico     47733
Otro                  24748
Posgrado               7387
Master                 3598
Doctorado               214
Name: educacion, dtype: int64